# Agents

### Loading OpenAI API Key

In [1]:
from dotenv import load_dotenv
import os

from langchain_core.output_parsers import StrOutputParser

# Load the .env file
load_dotenv(dotenv_path='../../.env')  # Specify the path to your .env file

# Access the environment variable
api_key = os.getenv('OPENAI_API_KEY')

# Check if the variable is loaded
if api_key or api_key == "":
    print("API key loaded successfully.")
else:
    print("Failed to load API key.")

API key loaded successfully.


### Initializing LLM

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage

llm = ChatOpenAI(
    model= "gpt-4o-mini",
    api_key= api_key,
)

response: AIMessage = llm.invoke("What are agents in the scope of LangChain? Are they lik agent Smith from movie Matrix? Please make you response sassy.")
print(response.content)

Oh honey, if you think agents in LangChain are anything like Agent Smith, you’re about to get a reality check tailor-made for a fabulous digital world! Agents in LangChain aren’t sinister, leather-clad programs trying to eradicate humanity—thank goodness! Instead, they are sophisticated constructs designed to handle tasks, make decisions, and interact with users through natural language processing.

Think of them more like your sassy sidekick who’s always got your back, organizing your thoughts, fetching data, and aiding in complex workflows while still managing to crack a joke or two. They operate autonomously by interpreting user intent, executing commands, and even engaging in multi-step actions seamlessly. 

So, while Agent Smith is busy trying to take over the Matrix, LangChain agents are all about enhancing productivity and giving you that extra sparkle in your tasks! Now, which one would you rather hang out with? I know my pick!✨


## ReAct Agents

In [3]:
import importlib.util

# Check if wikipedia is already installed
if importlib.util.find_spec("wikipedia") is None:
    print("Installing wikipedia package...")
    !pip install wikipedia
    print("Installation complete. You may need to restart the kernel.")
else:
    print("Wikipedia package is already installed.")

Wikipedia package is already installed.


In [21]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, AgentType, load_tools

# Define the tools
tools = load_tools(["wikipedia"], llm=llm)

# Define the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Run the agent
result = agent.run("How many people live in New York City?")
print(result)



> Entering new AgentExecutor chain...
I need to find the current population of New York City. I'll search for the most recent statistics on its population. 
Action: wikipedia
Action Input: "Population of New York City"
Observation: Page: Demographics of New York City
Summary: New York City is a large and ethnically diverse metropolis. It is the largest city in the United States, and has a long history of international immigration. The New York region continues to be by far the leading metropolitan gateway for legal immigrants admitted into the United States. The city is the geographical and demographic center of both the Northeast megalopolis and the New York metropolitan area, the largest metropolitan area in the U.S. by both population and urban area. With over 20.1 million people in its metropolitan statistical area and 23.5 million in its combined statistical area as of 2020, New York City is one of the world's most populous megacities.
The city and its metropolitan area are the 

### What is going on here? How does the agent above work?

When you run that cell you’ll see something like this:

> **Entering new AgentExecutor chain…**
> I need to find the current population of New York City. I’ll search for the most recent statistics on its population.
> **Action:** wikipedia
> **Action Input:** “Population of New York City”
> **Observation:**
> Page: *Demographics of New York City*
> Summary: New York City is a large and ethnically diverse metropolis… With over 20.1 million people in its metropolitan statistical area and 23.5 million in its combined statistical area as of 2020, New York City is one of the world’s most populous megacities.
>
> Page: *Demographic history of New York City*
> Summary: The racial and ethnic history of New York City has varied widely…
>
> Page: *New York City*
> Summary: New York, often called New York City (NYC), is the most populous city in the United States… With an estimated population in 2024 of **8,478,072**…
>
> **Thought:** I now have the most recent population estimate for New York City in 2024.
> **Final Answer:** The estimated population of New York City in 2024 is **8,478,072**.
>
> **Finished chain.**
> The estimated population of New York City in 2024 is 8,478,072.

---

#### In pseudo-code, the agent works like this:

```python
import openai

# 1. Define “tools” as simple functions
def wikipedia_search(query: str) -> str:
    return wiki_summary_for(query)

TOOLS = {
    "wikipedia": wikipedia_search,
}

# 2. System prompt explains the ReAct loop
SYSTEM = """
You are a helpful assistant. You have access to these tools:
{tool_list}

Use this format:
  Question: <the user’s question>
  Thought: <your reasoning>
  Action: <a tool name>
  Action Input: <tool argument>
  Observation: <tool result>
  …repeat Thought/Action/Action Input/Observation…
  Final Answer: <your answer>
""".strip()

# 3. The driver function
def react_agent(question: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM.format(tool_list=", ".join(TOOLS.keys()))},
        {"role": "user",   "content": f"Question: {question}"}
    ]
    while True:
        # 4. Ask the model
        resp = openai.ChatCompletion.create(model="gpt-4", messages=messages)
        chunk = resp.choices[0].message.content
        messages.append({"role": "assistant", "content": chunk})

        # 5. Stop if we have a final answer
        if "Final Answer:" in chunk:
            return chunk.split("Final Answer:")[-1].strip()

        # 6. Otherwise, pull out the next action
        action_line = find_line(chunk, "Action:")
        input_line  = find_line(chunk, "Action Input:")
        tool_name   = action_line.split(":", 1)[1].strip()
        tool_input  = input_line.split(":", 1)[1].strip()

        # 7. Call the tool
        observation = TOOLS[tool_name](tool_input)

        # 8. Feed back the result
        messages.append({"role": "assistant", "content": f"Observation: {observation}"})

# 9. Use it
answer = react_agent("How many people live in New York City?")
print(answer)
```

---

#### In simple terms

1. **Read your question.** “How many people live in New York City?”
2. **Think.** Decide you need Wikipedia.
3. **Call the tool.** “Action: wikipedia” → lookup “Population of New York City.”
4. **Observe the result.** Wikipedia says “8,478,072.”
5. **Think again.** You’ve got the data.
6. **Answer.** “The estimated population of New York City in 2024 is 8,478,072.”

It just repeats that loop—think, pick a tool, look it up, observe, think—until it gives you your final answer.
